# Bone Fracture Detection using YOLOv8

This Jupyter Notebook explores the application of deep learning for bone fracture detection using a comprehensive X-ray image dataset.  The dataset is specifically designed for computer vision projects and aims to facilitate the development and evaluation of automated bone fracture detection algorithms.

## About the Dataset

The dataset encompasses X-ray images categorized into several classes, each representing a specific type of bone fracture within the upper extremities. These classes include:

*   Elbow Positive
*   Fingers Positive
*   Forearm Fracture
*   Humerus Fracture
*   Shoulder Fracture
*   Wrist Positive

Each image is annotated with either bounding boxes or pixel-level segmentation masks, precisely indicating the location and extent of the detected fracture. These annotations are crucial for training and evaluating bone fracture detection algorithms, particularly object detection models.

This dataset provides a valuable resource for researchers and developers working on automated fracture detection. Its diverse range of fracture classes enables the training of robust models capable of accurately identifying fractures in various regions of the upper extremities. The ultimate goal of this dataset is to accelerate the development of computer vision solutions for automated fracture detection, thereby contributing to advancements in medical diagnostics and improved patient care.

**When using this dataset for your research, please cite it using the following DOI:** 10.13140/RG.2.2.14400.34569

**You can also find the dataset on ResearchGate:** [https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project](https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project)

## Imports

In [19]:
import tensorflow as tf
tf.config.list_physical_devices('GPU'), tf.__version__

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.18.0')

In [20]:

from dotenv import load_dotenv
from tensorflow import keras
import cv2

## Download datasets

In [21]:
# Loading kaggle keys
load_dotenv()

True

In [22]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/BoneFractureYolo8/train/


## Loading Images

In [25]:
CLASS_NAMES = ['elbow positive',
               'fingers positive',
               'forearm fracture',
               'humerus fracture',
               'humerus',
               'shoulder fracture',
               'wrist positive']

class_mapping = dict(zip(range(len(CLASS_NAMES)), CLASS_NAMES))
class_mapping

{0: 'elbow positive',
 1: 'fingers positive',
 2: 'forearm fracture',
 3: 'humerus fracture',
 4: 'humerus',
 5: 'shoulder fracture',
 6: 'wrist positive'}

In [26]:
from pathlib import Path

TRAIN_DIR = Path(cfg.DATASET_DIRS.TRAIN_DIR)
VALIDATION_DIR = Path(cfg.DATASET_DIRS.VALIDATION_DIR)
TEST_DIR = Path(cfg.DATASET_DIRS.TEST_DIR)

TRAIN_IMAGE_DIR = TRAIN_DIR / 'images'
TRAIN_LABELS_DIR = TRAIN_DIR / 'labels'

VALID_IMAGE_DIR = VALIDATION_DIR/'images'
VALID_LABELS_DIR = VALIDATION_DIR/'labels'

TEST_IMAGE = TEST_DIR/'images'
TEST_LABELS = TEST_DIR/'labels'

IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE


### Training and Validation Dataset setup

In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    resize_img = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    scaled_img = resize_img / 255.
    return tf.cast(scaled_img, tf.float32)


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    print(bbox)
    return {"images": image, "bounding_boxes": bounding_boxes}

In [28]:
# import os

# import numpy as np
# import tqdm
# from tqdm.notebook import tqdm


# class PrepareDataset:
#     def __init__(self, image_dir: str, label_dir: str) -> None:
#         """
#         Args:
#             image_dir (str): Path to the directory containing the images.
#             label_dir (str): Path to the directory containing the labels.
#         """
#         self.image_dir = image_dir
#         self.label_dir = label_dir

#     def get_dataset(self) -> tuple[list[str], list[int], list[np.ndarray]]:
#         """Loads and parses YOLOv8 labels.

#         Args:
#             None

#         Returns:
#             tuple[list[str], list[int], list[np.ndarray]]: A tuple containing:
#                 - A list of image paths.
#                 - A list of class ids.
#                 - A list of bounding boxes, where each bounding box is an array of 8 floats.
#         """
#         image_paths = []
#         class_ids = []
#         bboxes = []
#         for file_name in tqdm(os.listdir(self.image_dir)):
#             if file_name.endswith((".jpg", ".png")):
#                 imaga_file_path = os.path.join(self.image_dir, file_name)
#                 label_file_path = os.path.join(self.label_dir, file_name.replace(
#                     ".jpg", ".txt").replace(".png", ".txt"))

#                 if not os.path.exists(label_file_path):
#                     print(f"Label file not found for image: {imaga_file_path}")
#                     continue

#                 with open(label_file_path, 'r') as f:
#                     lines = f.readlines()

#                 if not lines:
#                     continue

#                 image_bboxes = []
#                 image_classes = []


#                 # 0 0.458736435546875 0.3806510419921875 0.3614540244140625 0.389472591796875 0.36892872265625 0.48237111328125 0.457112263671875 0.471341630859375 0.458736435546875 0.3806510419921875
#                 for line in lines:
#                     try:
#                         values = [float(value) for value in line.split()]
#                         class_id = int(values[0])
#                         coords = np.array(values[1:9]) # Extract the four corner points

#                         # Reshape coords to (1, 4) if it's a single box
#                         # if coords.size == 4:
#                         #     coords = coords.reshape(1, 4)  # Ensure it's a 2D array even for one box
#                                 # Reshape coords to (4, 2) if it's a single box
#                         if coords.size == 8:
#                             coords = coords.reshape(4, 2)  # Ensure it's a 2D array with four points

#                         # Calculate xmin, ymin, xmax, ymax
#                         x_coords = coords[:, 0]
#                         y_coords = coords[:, 1]
#                         xmin = np.min(x_coords)
#                         ymin = np.min(y_coords)
#                         xmax = np.max(x_coords)
#                         ymax = np.max(y_coords)
#                         image_bboxes.append(np.array([[xmin, ymin, xmax, ymax]]))
#                         image_classes.append(class_id)

#                     except Exception as e:
#                         print(f"[ERROR] - {e} in file {label_file_path} on line: {line}")
#                         continue
                    
#                 image_paths.append(imaga_file_path)
#                 class_ids.append(image_classes)
#                 bboxes.append(np.array(image_bboxes))  # Concatenate boxes for the image
                
#         # Ensure bounding boxes have the correct shape
#         bboxes = [np.array(bbox).reshape(-1, 4) for bbox in bboxes]
#         class_ids = [np.array(cls) for cls in class_ids]
#         return image_paths, class_ids, bboxes


In [29]:
from utils.prepare_dataset import PrepareDataset

preparer_train_ds = PrepareDataset(image_dir=TRAIN_IMAGE_DIR,
                                   label_dir=TRAIN_LABELS_DIR)
image_paths, class_ids, bboxes = preparer_train_ds.get_dataset()
len(image_paths), len(class_ids), len(bboxes)

0it [00:00, ?it/s]

(2088, 2088, 2088)

In [30]:
class_ids

[1,
 6,
 0,
 6,
 2,
 0,
 0,
 0,
 2,
 6,
 6,
 1,
 1,
 6,
 6,
 5,
 2,
 1,
 1,
 2,
 4,
 0,
 1,
 1,
 1,
 1,
 0,
 2,
 2,
 2,
 1,
 1,
 5,
 5,
 2,
 1,
 2,
 2,
 4,
 2,
 0,
 5,
 2,
 6,
 6,
 4,
 5,
 2,
 1,
 1,
 4,
 3,
 5,
 2,
 1,
 1,
 1,
 5,
 5,
 1,
 0,
 0,
 1,
 1,
 4,
 2,
 4,
 4,
 0,
 6,
 0,
 4,
 4,
 1,
 1,
 1,
 1,
 2,
 2,
 4,
 1,
 5,
 1,
 1,
 2,
 0,
 0,
 0,
 5,
 5,
 2,
 1,
 1,
 1,
 1,
 0,
 6,
 4,
 5,
 4,
 5,
 6,
 4,
 2,
 0,
 0,
 6,
 1,
 1,
 4,
 1,
 2,
 5,
 0,
 0,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 5,
 5,
 5,
 5,
 5,
 2,
 2,
 5,
 4,
 6,
 6,
 6,
 2,
 2,
 6,
 1,
 1,
 1,
 2,
 4,
 5,
 5,
 1,
 1,
 1,
 1,
 6,
 1,
 0,
 1,
 6,
 6,
 6,
 6,
 1,
 6,
 5,
 5,
 0,
 6,
 0,
 2,
 2,
 1,
 1,
 6,
 4,
 4,
 5,
 0,
 0,
 1,
 5,
 0,
 1,
 5,
 0,
 1,
 1,
 0,
 0,
 5,
 2,
 5,
 2,
 2,
 0,
 6,
 4,
 6,
 5,
 5,
 0,
 1,
 2,
 5,
 0,
 0,
 4,
 4,
 1,
 1,
 1,
 6,
 6,
 5,
 5,
 5,
 0,
 1,
 0,
 5,
 5,
 2,
 2,
 2,
 5,
 1,
 2,
 5,
 5,
 0,
 2,
 1,
 1,
 5,
 5,
 6,
 4,
 0,
 1,
 4,
 0,
 0,
 2,
 2,
 0,
 4,
 1,
 1,
 6,
 6,
 0,
 0,
 5,
 4,
 4,


In [31]:
bboxes

[array([[0.4404683 , 0.31309316, 0.7659444 , 0.55647546]], dtype=float32),
 array([[0.2908256 , 0.38371962, 0.41078433, 0.51536673]], dtype=float32),
 array([[0.4224264 , 0.4298628 , 0.60367703, 0.54083246]], dtype=float32),
 array([[0.41856024, 0.4094788 , 0.6055509 , 0.4855385 ]], dtype=float32),
 array([[0.365293  , 0.396875  , 0.5938265 , 0.55958796]], dtype=float32),
 array([[0.72946703, 0.3694884 , 0.8371946 , 0.49267924]], dtype=float32),
 array([[0.5997687 , 0.3987722 , 0.6947129 , 0.51465213]], dtype=float32),
 array([[0.55960584, 0.3642187 , 0.7474956 , 0.49238095]], dtype=float32),
 array([[0.5768778 , 0.60538375, 0.6844229 , 0.711113  ]], dtype=float32),
 array([[0.38905194, 0.4906167 , 0.7111077 , 0.591931  ]], dtype=float32),
 array([[0.7017998 , 0.41440684, 0.8375059 , 0.5022723 ]], dtype=float32),
 array([[0.54785424, 0.59742856, 0.7676488 , 0.7081445 ]], dtype=float32),
 array([[0.40979874, 0.47174376, 0.6072281 , 0.55683947]], dtype=float32),
 array([[0.5573702 , 0.34

In [32]:
bboxes_tensor = tf.ragged.constant(bboxes)
classes_tensor = tf.ragged.constant(class_ids)
image_paths_tensor = tf.ragged.constant(image_paths)
bboxes_tensor, classes_tensor, image_paths_tensor

(<tf.RaggedTensor [[[0.4404683, 0.31309316, 0.7659444, 0.55647546]],
 
  [[0.2908256, 0.38371962, 0.41078433, 0.51536673]],
 
  [[0.4224264, 0.4298628, 0.60367703, 0.54083246]],
 
  ...,
 
  [[0.35801572, 0.23696634, 0.63726234, 0.48981035]],
 
  [[0.35994813, 0.6314838, 0.5786385, 0.73620176]],
 
  [[0.40548456, 0.3172038, 0.6002938, 0.40337792]]]>,
 <tf.Tensor: shape=(2088,), dtype=int32, numpy=array([1, 6, 0, ..., 0, 4, 1], dtype=int32)>,
 <tf.Tensor: shape=(2088,), dtype=string, numpy=
 array([b'datasets/BoneFractureYolo8/train/images/image1_337_png.rf.ea9e9cb65cf9948ca00b6cb07bd84603.jpg',
        b'datasets/BoneFractureYolo8/train/images/image1_1189_png.rf.e538d3a229ada867f285dad84c25ddb8.jpg',
        b'datasets/BoneFractureYolo8/train/images/image2_818_png.rf.3388e2b6e41623534d485595d259afa2.jpg',
        ...,
        b'datasets/BoneFractureYolo8/train/images/image1_753_png.rf.fc30369df64af7966d1a4f54e6b39bc6.jpg',
        b'datasets/BoneFractureYolo8/train/images/image1_72_png

In [33]:
train_data = tf.data.Dataset.from_tensor_slices(
    (image_paths_tensor, classes_tensor, bboxes_tensor))

In [34]:
train_data

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None), RaggedTensorSpec(TensorShape([None, None]), tf.float32, 1, tf.int64))>

In [35]:
train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)\
    .shuffle(BATCH_SIZE * 4)\
    .ragged_batch(BATCH_SIZE, drop_remainder=True)


tf.RaggedTensor(values=Tensor("RaggedFromVariant/RaggedTensorFromVariant:1", shape=(None,), dtype=float32), row_splits=Tensor("RaggedFromVariant/RaggedTensorFromVariant:0", shape=(None,), dtype=int64))


In [36]:
for batch in train_ds.take(1):
    print("Before augmentation:")
    print("Images shape:", batch['images'].shape)
    print("Bounding boxes shape:", batch['bounding_boxes']['boxes'].shape)
    print("Classes shape:", batch['bounding_boxes']['classes'].shape)

Before augmentation:
Images shape: (16, None, None, 3)
Bounding boxes shape: (16, None, None)
Classes shape: (16,)


2025-02-12 07:40:49.042284: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [37]:
train_ds = train_ds.map(resizing_layer, num_parallel_calls=tf.data.AUTOTUNE)
for batch in train_ds.take(1):
    print("After augmentation:")
    print("Images shape:", batch['images'].shape)
    print("Bounding boxes shape:", batch['bounding_boxes']['boxes'].shape)
    print("Classes shape:", batch['bounding_boxes']['classes'].shape)

NameError: name 'resizing_layer' is not defined

In [ ]:
inputs = next(iter(train_ds.take(2)))
inputs["bounding_boxes"]

: 

: 

: 

: 

In [ ]:
valid_datasets = PrepareDataset(image_dir=VALID_IMAGE_DIR,
                                label_dir=VALID_LABELS_DIR)
valid_image_paths, valid_labels, valid_bboxes = valid_datasets.get_dataset()
len(valid_image_paths), len(valid_labels), len(valid_bboxes)

: 

: 

: 

: 

In [ ]:

val_data = tf.data.Dataset.from_tensor_slices((tf.ragged.constant(valid_image_paths),
                                               tf.ragged.constant(valid_labels),
                                               tf.ragged.constant(valid_bboxes)))
val_ds = val_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)

: 

: 

: 

: 

In [ ]:
resizing = keras_cv.layers.JitteredResize(
    target_size=(IMG_SIZE, IMG_SIZE),
    scale_factor=(0.75, 1.3),
    bounding_box_format="xyxy",
)
val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

: 

: 

: 

: 

In [ ]:
def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

: 

: 

: 

: 

In [ ]:
train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

: 

: 

: 

: 

In [ ]:
import sigstore
print(sigstore.__version__)

: 

: 

: 

: 

In [ ]:

backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone_coco"  # We will use yolov8 small backbone with coco weights
)

: 

: 

: 

: 

In [ ]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    fpn_depth=1,
)

: 

: 

: 

: 

: 

: 

: 

: 